# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
#Your code here; import the necessary packages
import sqlite3 
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

In [3]:
#Your code here; create the connection and cursor

## Select the Average Number of Orders on a Product by Product Basis

In [4]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""   select avg(numOrders) from
                    (select productCode,
                                 count(distinct orderNumber) as numOrders
                                 from products
                                 join orderdetails
                                 using(productCode)
                                 group by 1)
                         
          """
         ).fetchall()

[(27.486238532110093,)]

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [5]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       where customerNumber in (select customerNumber from orders where orderDate = '2003-01-31')""").fetchall()

[('141', 'Freyre', 'Diego ')]

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [6]:
#Your code here
import pandas as pd
d1 = pd.DataFrame(c.execute("""
            select productName as p, count(orderNumber) as num_sold, sum(quantityOrdered) tol_units
            from products 
            join orderdetails using (productCode) group by productName
            order by tol_units desc""").fetchall())
d1.columns = [x[0] for x in c.description]

<img src="images/Database-Schema.png" width="600">


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [7]:
#Your code here
d2 = pd.DataFrame(c.execute("""
            select productName as p, count(distinct customerNumber) as num_people
            from products 
            join orderdetails using (productCode) 
            join orders using (orderNumber)
            group by productName
            order by num_people desc""").fetchall())
d2.columns = [x[0] for x in c.description]

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.


<img src="images/Database-Schema.png" width="600">

In [10]:
#Your code here
d3 = pd.DataFrame(c.execute("""
            select distinct employeeNumber as em, officeCode as oc,
            of.city as coo, firstName, lastName
            
            from employees as em
            join offices as of
            using (officeCode) 
            join customers as cus
            on em.employeeNumber = cus.salesRepEmployeeNumber
            join orders as o
            using (customerNumber)
            join orderdetails as od
            using (orderNumber)
            join products as pd
            using (productCode)
            
            where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20
                                         )
            """))
d3.columns = [x[0] for x in c.description]
d3


TypeError: data argument can't be an iterator

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [13]:
#Your code here
c.execute("""select employeeNumber,
                    firstName,
                    lastName,
                    count(distinct customerNumber)
                    from employees e
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    group by 1
                    having AVG(creditLimit) > 15000;
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,firstName,lastName,count(distinct customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!